In [1]:
import numpy as np
from IPython.display import HTML

# A step to step guide to understand attention mechanisms

In [2]:
image_path = '../latex/figures/MultiheadAttention.svg'
display(HTML(f'<img src="{image_path}" style="width: 100%;">'))

In this notebook, we aim to provide a step-by-step elucidation of the Multihead-Attention mechanism, utilizing generated visualizations and concrete numerical values, as a supplementary elaboration to the explanations presented in the project report. In order to render these explanations reasonably comprehensible, it is imperative that we temporarily depart from the high-dimensional visualization presented above (which mirrors the dimensionalities employed in the 'Attention Is All You Need' paper and maintains an input sequence length of $n=7$, ensuring consistency with the example *'I try to understand the Transformer architecture'* in the report).

Let us imagine that we are left with only one sentence of length $n=3$ to elucidate the attention mechanism, namely, *'I love Transformers'*. Furthermore, we will assume a model dimensionality of 4 instead of 512 dimensions, and the calculations will be divided into 2 attention heads. This automatically determines the size or dimensionality of an individual attention head. All of these parameters are set in the following code cell:

In [3]:
# Set parameters
# Number of attention heads
h = 2

# Dimensions
d_model = 4
d_k = d_v = d_model // h

# Sequence length
n = len("I love Transformers".split())

As we progress further in this notebook, we will now proceed step by step from left to right through the upper visualization.

## Step 1: Initialize input matrices
First, we need to initialize the gray-colored encoder input of size $(n, d_{model})$. Since this notebook does not directly build upon the two preceding architectural components of the Transformer - namely, the **input embedding** and **positional encoding** - we will initialize this matrix with arbitrary values ranging from 0 to 9. Specifically, we will designate that the first row of the matrix corresponds to a 4-dimensional representation of the word 'I', the second row to the representation of the word 'love', and the third row to the representation of the word 'Transformers' (we can readily assume this for now, as we have omitted the initial Transformer steps in this context).

The three matrices $Q$ (pink), $K$ (orange), and $V$ (yellow) are initially just three copies of this encoder input matrix:

In [4]:
# Input matrix
np.random.seed(0)  # Set seed for reproducibility
input_matrix = np.random.randint(10, size=(n, d_model))  # 3x4 matrix with random values between 0 and 9

# Copies of the input matrix for Q, K and V
Q = input_matrix.copy()
K = input_matrix.copy()
V = input_matrix.copy()

Let's take a look at how our randomly generated representations for the input sequence *'I love Transformers'* appear:

In [5]:
# Printing
print(f"Input Matrix = Q = K = V: \n{input_matrix}")

Input Matrix = Q = K = V: 
[[5 0 3 3]
 [7 9 3 5]
 [2 4 7 6]]


## Step 2: Initialize parameter matrices
As evident from the visualization, matrices $Q$, $K$, and $V$ are subjected to matrix multiplication with their corresponding parameter or projection matrices, denoted as $W_i^Q$, $W_i^K$, and $W_i^V$, respectively. The index 'i' signifies the specific attention head under consideration.

But how to initialize the parameter matrices?</br>
To ensure more robust training, the initialization of these parameter matrices is a crucial step. In practice, weight matrices are initialized with carefully chosen initial values. In this context, we employ the Xavier Uniform Initialization method, introduced in 2010 in the paper titled "[Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)" by Xavier Glorot and Yoshua Bengio. This initialization strategy ensures that the weights are distributed in such a way that the variance of signals is preserved as they propagate through the layers of a network. This preservation of signal variance helps prevent the issues of vanishing or exploding gradients during training, a particularly important consideration in deep architectures such as the Transformer model. It enables more efficient and effective weight adjustments during the learning process.

In the following code cell, we initialize the weight matrices $W_i^Q$, $W_i^K$, and $W_i^V$ independently for each attention unit, representing the different heads of the Multihead Attention mechanism.

In [6]:
def xavier_uniform_init(input_size, output_size) -> np.ndarray:
    limit = np.sqrt(6 / (input_size + output_size))
    return np.random.uniform(-limit, limit, (input_size, output_size))
# xavier uniform initialization of the weight matrices
W_Q = np.array([xavier_uniform_init(d_model, d_k) for _ in range(h)])
W_K = np.array([xavier_uniform_init(d_model, d_k) for _ in range(h)])
W_V = np.array([xavier_uniform_init(d_model, d_v) for _ in range(h)])

As we previously defined two attention heads at the beginning of this notebook, we consequently have two weight matrices each for Queries, Keys, and Values, as illustrated below for $W^Q$:


In [7]:
W_Q

array([[[-0.88657405, -0.45468741],
        [-0.04466977,  0.62433746],
        [-0.04004566, -0.21443041],
        [ 0.67215753, -0.32520768]],

       [[ 0.29634374, -0.26351692],
        [ 0.91431032, -0.71929844],
        [ 0.74017452, -0.05278391],
        [ 0.6018215 ,  0.04095496]]])

In [8]:
assert W_Q[0].shape == W_Q[1].shape
W_Q[0].shape

(4, 2)

As we can see, the two generated weight matrices have a dimensionality of $(4, 2)$, which corresponds to the $d_{model}$ rows and $d_v=d_k$ columns. Since we have an additional dimension for the number of attention heads, in the visualization, we observe third-level tensors $W_Q$, $W_K$, and $W_V$. In our specific example, these tensors would have a dimensionality of $(4, 2, 2)$. We can imagine that we stack the individual matrices together along the third dimension.

*Note:*</br>
As evident from the notation, the distinction between matrices $W_i^Q$ and tensors $W^Q$ is crucial for understanding the architecture of the Multihead Attention mechanism.
* The matrix $W_i^Q$ refers to the weight matrix associated with the $i$-th attention head for the query component, represented as a 2D matrix
* In constrast, $W^Q$ denotes the entire collection of these weight matrices for all heads, assembled into a 3D tensor. Each "slice" of this tensor corresponds to a specific head's weights, $W_i^Q$, thus encapsulating the multi-head aspect of the mechanism within a single tensorial structure.

## Step 3: Calculating the projected matrices $Q_i^\prime$, $K_i^\prime$ and $V_i^\prime$
Here we will obtain the projected matrices $Q_i^\prime$, $K_i^\prime$ and $V_i^\prime$ that build the tensors $Q^\prime$, $K^\prime$ and $V^\prime$ shown in the visualization. Let's first take a look at how we get the new query representations for the two attention heads by applying $Q \times W_i^Q$ for each head:

In [9]:
Q_prime = np.array([np.dot(Q, W) for W in W_Q])
Q_prime

array([[[-2.53653461, -3.89235132],
        [-3.36739554,  0.16689562],
        [ 1.80079842, -1.86428392]],

       [[ 5.50770678, -1.35307145],
        [15.53283014, -8.27188133],
        [13.0420794 , -3.52798521]]])

This tensor therefore consists of the two matrices $Q_1^\prime$ und $Q_2^\prime$ that have the dimensionalitity $(n,d_k)$:

In [10]:
assert Q_prime[0].shape == Q_prime[1].shape
Q_prime[0].shape

(3, 2)

Similarly we obtain the matrices $K_i^\prime$ and $V_i^\prime$ for $i=1,2$:

In [11]:
K_prime = np.array([np.dot(K, W) for W in W_K])
V_prime = np.array([np.dot(V, W) for W in W_V])
for i, (K_i_prime, V_i_prime) in enumerate(zip(K_prime, V_prime)):
    print(f"K_{i+1}_prime: \n{K_i_prime}")
    print(f"V_{i+1}_prime: \n{V_i_prime}")

K_1_prime: 
[[ 3.18209156 -2.04023375]
 [ 5.26836762 -1.73965563]
 [ 4.67550023 -8.09978892]]
V_1_prime: 
[[ 0.34381182  1.1983878 ]
 [ 6.06952461 -6.1297034 ]
 [ 5.6468306  -2.31171686]]
K_2_prime: 
[[-0.41383514 -2.08497727]
 [-6.48627938 -4.77384381]
 [-8.23726599 -1.59698938]]
V_2_prime: 
[[ 4.22924766  4.64235554]
 [ 2.69400362 -2.69442861]
 [ 3.31644301  4.78472233]]


Attention should be drawn to two things:</br>
* The original matrices $Q$, $K$ and $V$ had dimensions of $(n,d_{model})$, where $d_{model}$ represents the embedding size for each word in the sequence. After projection, the resulting matrices $Q_i^\prime$, $K_i^\prime$ and $V_i^\prime$ have dimensions $(n, d_k)$ or $(n, d_v)$, which are the reduced dimensions for each attention head. For our specific example *"I love Transformers"*, this means that the original 4-dimensional vector representations of the words are transformed into two separate 2-dimensional representations for each attention head. This allows different attention heads to capture different aspects of the same word. For instance, one head might focus on the syntactic role of *"I"*, whie another might attend to its semantic context. By doing so, the model can parallelly process and integrate diverse information streams, enhancing its ability to discern and utilize complex dependencies within the text.
* While the initial matrices $Q$, $K$ and $V$ were identical copies derived from the input encoding, representing the same information for each token, the projection process creates distinct matrices $Q_i^\prime$, $K_i^\prime$ and $V_i^\prime$. This divergence is deliberate. By applying different weight matrices $W_i^Q$, $W_i^K$ and $W_i^V$ for each head, the model generates varied representations for the queries, keys and values.
  
This means that in the Multihead Attention architecture, diversity emerges on two levels. Firstly, by splitting the original word representations into multiple vectors, we enable different attention heads ($h$) to focus on diverse aspects of the data, introducing the first level of diversity through the multi-head structure itself. Additionally, applying distinct weight matrices ($W_i^Q$, $W_i^K$ and $W_i^V$) to the identical input matrices $Q$, $K$ and $V$ creates a second level of diversity. Each projected matrix $Q_i^\prime$, $K_i^\prime$ and $V_i^\prime$ now encodes different information, even though they originate from the same input data. This allows the model to recognize and respond to a broader spectrum of information within the data, crucial for understanding complex dependencies in the text.

#### Intermediate stop - next steps
In the next steps, the attention function is applied according to the formula $$Attention(Q,K,V)= softmax(\frac{QK^T}{\sqrt{d_k}})V$$ as described in "[Attention Is All You Need](https://arxiv.org/abs/1706.03762)".</br>
Since we are dealing with multi-head attention, we calculate the whole thing for the queries, keys and values of each individual attention head according to $$MultiHead(Q,K,V)=Concat(head_1,...,head_h)W^O$$ where $$head_i=Attention(QW_i^Q,KW_i^K,VW_i^V)$$ We then append the results of the individual attention heads to each other again and project the concatenated matrix once more using the projection matrix $W^O$ to obtain the final attention values. However, we proceed step by step. In practice, this is all done in parallel so that the arithmetic operations are as efficient as possible.

## Step 4: Calculating the scaled attention scores
In this step, we calculate the attention scores, which determine how much focus to allocate to each word in the input sequence for generating the final output representation. These scores are obtained by taking the **dot product** of the projected query matrix $Q_i^\prime$ with the transpose of the projected key matrix $K_i^\prime$ for each attention head. The result of this calculation is represented in the visualization by the red tensor of dimensions $(n, n, h)$. 

To ensure numerical stability and avoid gradients that are too small during backpropagation, we **scale** the attention scores by dividing by the square root of the key dimensionality, $\sqrt{d_k}$. This scaling process adjusts the representations of *"I love Transformers"*, not just reflecting the individual words but also their contextual relationship within the sequence. Consequently, the model directs its focus more effectively.

The raw, scaled attention scores from each head are then passed through a **softmax** function to normalize them into probabilities, represented as a light brown tensor in our visualization. The softmax function ensures that each row of the attention scores — corresponding to the attention distribution from one query word to all key words in a single attention head — sums to 1. This normalization allows the model to assign a probability distribution over all words, quantifying the relative importance each word has on another, thereby enabling the model to attend selectively to the most pertinent information for each word in the sequence.

In [12]:
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)
scaled_attention_scores = np.array([softmax(Q_prime[i].dot(K_prime[i].T) / np.sqrt(d_k)) for i in range(h)])
scaled_attention_scores

array([[[8.32207466e-07, 8.62661112e-09, 9.99999159e-01],
        [9.79261626e-01, 7.06124878e-03, 1.36771253e-02],
        [5.06701339e-05, 4.85740226e-04, 9.99463590e-01]],

       [[9.99999999e-01, 7.02243021e-10, 3.67172510e-14],
        [1.00000000e+00, 7.32297340e-23, 2.77013209e-39],
        [1.00000000e+00, 3.91078473e-22, 1.37240079e-32]]])

In [13]:
scaled_attention_scores[0].sum(axis=-1)

array([1., 1., 1.])

*Notice:*</br>
Each attention head computes its own set of scaled scores, potentially focusing on different parts of the input sequence. 

## Step 5: Calculate the weighted attention scores
In the next code cell, we compute the weighted attention scores by multiplying the normalized attention scores from Step 4 with the yellow visualized value matrix $V_i^\prime$ for each attention head. This operation essentially applies the calculated attention to the values.

In [14]:
weighted_scores = np.array([scaled_attention_scores[i].dot(V_prime[i]) for i in range(h)])
weighted_scores

array([[[ 5.64682619, -2.31171397],
        [ 0.45677255,  1.09863418],
        [ 5.64676721, -2.31339355]],

       [[ 4.22924766,  4.64235554],
        [ 4.22924766,  4.64235554],
        [ 4.22924766,  4.64235554]]])

Each row of the resulting tensor (the dark brown one of dimensionality $(n, d_v, h)$ in the visualization) represents the attention-weighted representations of a word in the sequence, informed by its context. Specifically, the first row reflects how the first word 'attends to' every other word, combining the values from $V_i^\prime$ weighted by the attention scores. This produces a contextually enriched representation of the first word based on its relevance to the rest of the sequence. In essence, each row in the weighted score matrix for each head is a composite representation that integrates the contextual information as determined by the attention mechanism across all words.

## Step 6: Concatenate the weighted attention scores
In Step 6 of the Transformer model, we concatenate the weighted attention scores across all attention heads to form a unified representation. This concatenation is performed because each attention head may focus on different aspects of the input sequence, and by combining them, we capture a more comprehensive representation of the sequence.

The concatenation is done along the dimension representing the number of heads (usually the last dimension of the tensor). In NumPy, this is achieved using `np.concatenate`, specifying the axis along which the concatenation should occur. Here, axis=1 is used because we are concatenating the second dimension of the tensor, which aligns with the attention heads in a typical implementation where each head's output is a separate matrix in an array. After concatenation, the resulting tensor has information from all heads merged along the specified axis, ready for further processing. Thus, from the third-level tensor with dimensionality $(n, d_v, h)$, as illustrated in the visualization, a matrix with dimensionality $(n, d_v \cdot h)$ is obtained. Since $d_{model} = d_v \cdot h$, we consequently obtain a matrix that has the same size as our original gray input matrix, as previously depicted:

In [15]:
concatenated_values = np.concatenate(weighted_scores, axis=1)
concatenated_values

array([[ 5.64682619, -2.31171397,  4.22924766,  4.64235554],
       [ 0.45677255,  1.09863418,  4.22924766,  4.64235554],
       [ 5.64676721, -2.31339355,  4.22924766,  4.64235554]])

In [16]:
concatenated_values.shape

(3, 4)

## Step 7: Project the final matrix
Lastely, the concatenated output from all attention heads is projected through the weight matrix $W^O$:

In [17]:
W_O = xavier_uniform_init(h * d_v, d_model)
output = concatenated_values.dot(W_O)
output

array([[ 7.3765234 ,  6.13875477,  3.44813173, -0.03779159],
       [ 3.6536207 ,  4.44609421,  5.25015372, -0.89010674],
       [ 7.3761135 ,  6.13921767,  3.44763211, -0.03725722]])

This projection serves to synthesize the information from all heads, effectively allowing the model to combine the different feature representations captured by each head into a unified output. This step is essential for the Transformer architecture, as it distills the broad array of features into a format suitable for further processing by subsequent layers of the network. The result is an output matrix that maintains the original embedding dimensionality $d_{model}$​ while encapsulating a rich, multi-perspective understanding of the input sequence. This matrix now contains the final attention values!